In [ ]:
%run helpers.py
%run classification.py
%run transformations.py
%run model.py
%run prediction.py

In [ ]:
import argparse
import os
import sys
import classification
import helpers
import transformations as tr
import model
import prediction
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim


In [ ]:
def parse_cmdline_arguments():
    """Parse command line arguments"""
    parser = argparse.ArgumentParser(description='Image classification...')
    parser.add_argument('-d', '--dir_path', type=str, default=(os.getcwd() + '\\data\\part1'),
                        help='Full path of dataset containing images')
    parser.add_argument('-t', '--test_images_path', type=str, default=(os.getcwd() + '\\data\\part1\\bristol\\person_2'),
                        help='Full path of dataset containing test images')
    parser.add_argument('--csv_file_dir', type=str, default=(os.getcwd() + '\\data\\part2'),
                        help='csv file directory having bbox for images')
    parser.add_argument('-m', '--pretrained_model', type=str.lower,
                        choices=['resnet34', 'resnet50'],
                        default='resnet34', help='Pretrained base model name')
    parser.add_argument('-e', '--epochs', type=int, default=3,  help='No. of epochs')
    parser.add_argument('--activation', type=str.lower, choices=['relu'],
                        default='relu', help='Activation function')
    parser.add_argument('-bs', '--batch_size', type=int, default=4,
                        help='Batch size')
    parser.add_argument('-lr', '--learning_rate', type=float, default=0.001,
                        help='Learning rate')
    # parser.add_argument('--dropout', type=float, default=0.0,
                        # help='Dropout fraction')
    # parser.add_argument('--augment', action='store_true',
                        # help='Perform image augmentation')
    # parser.add_argument('--ckpt_period_and_path', type=str, nargs=2,
                        # default=[1000, 'training/cp-{epoch:04d}.ckpt'],
                        # help='Save checkpoints every <period> steps at <dir>')
    parser.add_argument('-v', '--verbose', action="store_true",
                        help='Increase output verbosity')

    return parser.parse_args(args=[])


In [ ]:
def main():
    args = parse_cmdline_arguments()
    
    # dir_path =  args.dir_path if args.dir_path else (os.getcwd() + '\\data\\part1')
    helpers.check_path_exists(args.dir_path, 'train images directory')
    
    df_player_info, teams_dic, players_dic = helpers.generate_player_info(args.dir_path, check_invalid_images_flag=False)

    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
    train_transform = transforms.Compose([tr.Resize(224, 224), tr.ToTensor(), tr.Normalization(mean,std)])
    test_transform = transforms.Compose([tr.Resize(224, 224), tr.ToTensor(), tr.Normalization(mean,std)])

    # Split the data in training and validation setl
    data = helpers.split_data(df_player_info, test_size=0.2)

    data['train'] = classification.Classification(player_info=data['train'], transform=train_transform)
    data['valid'] = classification.Classification(player_info=data['valid'], transform=train_transform)
    
    train_dataloader = DataLoader(dataset=data['train'], batch_size=args.batch_size, shuffle=True, num_workers=0)
    valid_dataloader = DataLoader(dataset=data['valid'], batch_size=args.batch_size, shuffle=False, num_workers=0)
    dataloaders = {phase:train_dataloader if phase == 'train' else valid_dataloader for phase in ['train', 'valid']}

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device("cpu")
    # Setting model and moving to device
    model_CNN = model.Model(True, args.pretrained_model, len(teams_dic), len(players_dic)).to(device)
    # For multilabel output: Same criterion here for 'teams' and 'players', but can change as required
    criterions = {phase:nn.CrossEntropyLoss() if phase == 'teams' else nn.CrossEntropyLoss() for phase in ['teams', 'players']}
    optimizer = optim.SGD(model_CNN.parameters(), lr=args.learning_rate, momentum=0.9)

    model_CNN = model_CNN.train_model(model_CNN, dataloaders, criterions, optimizer, device)
    print('\ndone training...')
    torch.save(model_CNN.state_dict(), 'model_CNN.ckpt')
    model_CNN.load_state_dict(torch.load('model_CNN.ckpt'))

    predict = prediction.Prediction(model_CNN, test_transform)
    
    # test_images_path = args.test_images_path if args.test_images_path else (os.getcwd() + '\\data\\part1\\bristol\\person_2')
    helpers.check_path_exists(args.test_images_path, 'test images directory')
    # predict.predict_images_in_directory(args.test_images_path, check_invalid_images_flag=False)
    
    # csv_file_dir = args.test_images_path if args.test_images_path else (os.getcwd() + '\\data\\part1\\bristol\\person_2')
    predict.predict_and_color_section_of_images(args.csv_file_dir)




if __name__ == '__main__':
    main()